###  Created by Luis Alejandro (alejand@umich.edu)

## Model Selection
Model selection is choosing between different complexity or flexibility to fit/explain your data.
#### Complexity Controlling Parameters
* Not used to fit the data
* Usually called "hyperparameters" (i.e., in Bayesian models parameters of the prior for instance)

#### Selecting best complexity parameters
Selecting the best complexity parameters usually consist of exploring different combinations of the hyperparameters to explain your data. More technically this search consist of:
* An estimator (regressor or classifier )
* A parameter space
* A method for searching or sampling candidates
* A cross-validation scheme
* A score function

In [1]:
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import numpy as np
import pandas as pd
import time
import sys
sys.path.append('../')
import utils.reports as rp

### Dataset 1
Performs model selection of the following hyperparameters applied to the bank dataset (customers leaving):
* Penalty (l1 or l2)
* C (Inverse of regularization)

This is perform using cross-validation and a grid search using [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).


In [2]:
# Loads dataset from file
dataset = pd.read_csv('../../datasets/classification/bank_exiting.csv')
predictors = dataset.iloc[:,3:-1].values
responses = dataset.iloc[:,-1].values
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
# Encoding the gender feature
print(dataset['Gender'].unique())
print(predictors[1,:])
encoder = LabelEncoder()
predictors[:,2] = encoder.fit_transform(predictors[:,2]) # only 0 or 1 after this (just one column needed)
print(predictors[1,:])

['Female' 'Male']
[608 'Spain' 'Female' 41 1 83807.86 1 0 1 112542.58]
[608 'Spain' 0 41 1 83807.86 1 0 1 112542.58]


In [4]:
# Encoding the country feature
print(predictors[1,:])
ct = ColumnTransformer([('country_category', OneHotEncoder(categories='auto', drop='first'),[1])], remainder='passthrough')
predictors = ct.fit_transform(predictors)
print(predictors[1,:])
X,X_holdout,y,y_holdout = train_test_split(predictors, responses, test_size = 0.2)

[608 'Spain' 0 41 1 83807.86 1 0 1 112542.58]
[0.0 1.0 608 0 41 1 83807.86 1 0 1 112542.58]


In [5]:
# This step should not be here since we will leak info into the validation sets!!!
ct = ColumnTransformer([('normalizer', StandardScaler(),[2,4,5,6,7,10])],remainder='passthrough')
X = ct.fit_transform(X)
X_holdout = ct.transform(X_holdout)
X[0,:]

array([1.5397174116277985, -0.9499994057663559, -1.040467106253047,
       -1.231796281195587, 0.8108355785714346, 1.1671011270196134, 0.0,
       0.0, 0, 0, 0], dtype=object)

In [6]:
# Performs grid search
start = time.perf_counter()
C_choices = [0.0001, 0.1, 0.2, 0.5, 0.7, 1]
penalty_choices = ['l1','l2']

hyperparams = [{
    'penalty': penalty_choices,
    'C': C_choices
}]

mdl = LogisticRegression(max_iter=200)
validator = GridSearchCV(mdl, cv=3, param_grid=hyperparams, scoring='accuracy', n_jobs=-1,verbose = 1)
validator.fit(X,y)
end = time.perf_counter()
print('Elapsed time: ', end - start, ' seconds\n')
rp.report_grid_search(validator.cv_results_)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Elapsed time:  1.8673037000000001  seconds

Model with rank: 1
Mean validation score: 0.811 (std: 0.002)
Parameters: {'C': 0.1, 'penalty': 'l2'}

Model with rank: 2
Mean validation score: 0.810 (std: 0.002)
Parameters: {'C': 0.2, 'penalty': 'l2'}

Model with rank: 3
Mean validation score: 0.809 (std: 0.002)
Parameters: {'C': 0.5, 'penalty': 'l2'}



[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:    1.7s finished


In [7]:
# Perform evaluation in the holdout set
y_pred = validator.predict(X_holdout)
rp.report_classification(y_holdout,y_pred,title='Holdout')

Holdout (Metrics): 

Accuracy:  0.80
F1 Score:  0.31
Recall:  0.21
Precision:  0.58

Confusion Matrix:
 [[1523   63]
 [ 328   86]]


### Dataset 2
Performs model selection of the following hyperparameters applied to the wine dataset:
* Penalty (l1 or l2)
* C (Inverse of regularization)
* Max # of iterations

This is perform using cross-validation and a grid search using [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Notice that no standarization is applied in this case

In [8]:
# Load dataset
dataset = datasets.load_wine()
predictors = dataset.data
responses = dataset.target
X,X_holdout,y,y_holdout = train_test_split(predictors, responses, test_size = 0.2,stratify=responses)

In [9]:
# Performs grid search
start = time.perf_counter()
C_choices = [0.0001, 0.1, 0.2, 0.5, 0.7, 1]
iter_choices = [10, 20, 50, 100, 300]

hyperparams = [{
    'penalty': ['l1'],
    'C': C_choices,
    'max_iter': iter_choices,
    'solver': ['saga']
},    
{
    'penalty': ['l2'],
    'C': C_choices,
    'max_iter': iter_choices,
    'solver': ['lbfgs']
}]

mdl = LogisticRegression(multi_class='multinomial')
validator = GridSearchCV(mdl, cv=5, param_grid=hyperparams, scoring='accuracy', n_jobs=-1,verbose = 1,iid = False)
validator.fit(X,y)
end = time.perf_counter()
print('Elapsed time: ', end - start, ' seconds\n')
rp.report_grid_search(validator.cv_results_)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.0s


Elapsed time:  0.9235902999999999  seconds

Model with rank: 1
Mean validation score: 0.950 (std: 0.028)
Parameters: {'C': 0.7, 'max_iter': 100, 'penalty': 'l2', 'solver': 'lbfgs'}

Model with rank: 1
Mean validation score: 0.950 (std: 0.028)
Parameters: {'C': 0.7, 'max_iter': 300, 'penalty': 'l2', 'solver': 'lbfgs'}

Model with rank: 1
Mean validation score: 0.950 (std: 0.028)
Parameters: {'C': 1, 'max_iter': 300, 'penalty': 'l2', 'solver': 'lbfgs'}



[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:    0.8s finished
D:\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning
D:\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [10]:
# Perform evaluation in the holdout set
y_pred = validator.predict(X_holdout)
rp.report_classification(y_holdout,y_pred,avg='macro',title='Holdout')

Holdout (Metrics): 

Accuracy:  0.97
F1 Score:  0.97
Recall:  0.97
Precision:  0.98

Confusion Matrix:
 [[12  0  0]
 [ 0 14  0]
 [ 0  1  9]]


### Dataset 3
Performs model selection of the following hyperparameters applied to the wine dataset:
* Penalty (l1 or l2)
* C (Inverse of regularization)
* Max # of iterations

This is perform using cross-validation and a grid search using [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Since we want to apply standarization, we must use a [Pipeline](https://scikit-learn.org/stable/modules/compose.html#pipeline) to correctly standarize and evaluate the models during the cross-validation.

In [11]:
# Load dataset
dataset = datasets.load_wine()
predictors = dataset.data
responses = dataset.target
X,X_holdout,y,y_holdout = train_test_split(predictors, responses, test_size = 0.2)

In [12]:
# Performs grid search
start = time.perf_counter()

sc = StandardScaler()
clf = LogisticRegression(multi_class='ovr')
estimators = [('normalizer', sc), ('classifier', clf)]
pipe = Pipeline(estimators)

C_choices = [0.0001, 0.1, 0.2, 0.5, 0.7, 1]
iter_choices = [10, 20, 50, 100, 300]


hyperparams = [{
    'classifier__penalty': ['l1'],
    'classifier__C': C_choices,
    'classifier__max_iter': iter_choices,
    'classifier__solver': ['saga']
},    
{
    'classifier__penalty': ['l2'],
    'classifier__C': C_choices,
    'classifier__max_iter': iter_choices,
    'classifier__solver': ['lbfgs']
}]

validator = GridSearchCV(pipe, cv=5, param_grid=hyperparams, scoring='accuracy', n_jobs=-1,verbose = 1,iid = False)
validator.fit(X,y)
end = time.perf_counter()
print('Elapsed time: ', end - start, ' seconds\n')
rp.report_grid_search(validator.cv_results_)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.1s


Elapsed time:  0.6007431999999993  seconds

Model with rank: 1
Mean validation score: 0.986 (std: 0.029)
Parameters: {'classifier__C': 0.5, 'classifier__max_iter': 10, 'classifier__penalty': 'l1', 'classifier__solver': 'saga'}

Model with rank: 1
Mean validation score: 0.986 (std: 0.029)
Parameters: {'classifier__C': 0.5, 'classifier__max_iter': 20, 'classifier__penalty': 'l1', 'classifier__solver': 'saga'}

Model with rank: 1
Mean validation score: 0.986 (std: 0.029)
Parameters: {'classifier__C': 0.5, 'classifier__max_iter': 50, 'classifier__penalty': 'l1', 'classifier__solver': 'saga'}

Model with rank: 1
Mean validation score: 0.986 (std: 0.029)
Parameters: {'classifier__C': 0.5, 'classifier__max_iter': 100, 'classifier__penalty': 'l1', 'classifier__solver': 'saga'}

Model with rank: 1
Mean validation score: 0.986 (std: 0.029)
Parameters: {'classifier__C': 0.5, 'classifier__max_iter': 300, 'classifier__penalty': 'l1', 'classifier__solver': 'saga'}

Model with rank: 1
Mean validation

[Parallel(n_jobs=-1)]: Done 277 out of 300 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:    0.5s finished
D:\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning
D:\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
D:\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
D:\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [13]:
# Perform evaluation in the holdout set
y_pred = validator.predict(X_holdout)
rp.report_classification(y_holdout,y_pred,avg='macro',title='Holdout')

Holdout (Metrics): 

Accuracy:  1.00
F1 Score:  1.00
Recall:  1.00
Precision:  1.00

Confusion Matrix:
 [[14  0  0]
 [ 0 15  0]
 [ 0  0  7]]
